# 載入模組

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 取得網頁

In [2]:
url = 'https://mops.twse.com.tw/mops/web/ajax_t05st03'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36'}
payload = {
    'encodeURIComponent':'1',
    'step':'1',
    'firstin':'1',
    'off':'1',
    'queryName':'co_id',
    'inpuType':'co_id',
    'TYPEK':'all',
    'co_id':'2330',
    }


resp = requests.post(url, data=payload,headers = headers)

# 解析網頁

In [3]:
soup = BeautifulSoup(resp.text, 'html.parser')

# 取得全部有效table
table = soup.find_all('table',{'class': 'hasBorder'})
table[1]

<table class="hasBorder" width="95%">
<th class="dColor nowrap" style="text-align:left !important;">本公司採</th>
<td class="lColor"> </td><th class="dColor nowrap" colspan="3" style="text-align:left !important;">月制會計年度（空白表曆年制）</th>
<tr>
<th class="dColor nowrap" style="text-align:left !important;">本公司於</th>
<td class="lColor"> </td><th class="dColor nowrap" style="text-align:left !important;">之前採</th>
<td> </td><th class="dColor nowrap" style="text-align:left !important;">月制會計年度</th>
</tr>
<tr>
<th class="dColor nowrap" colspan="2" style="text-align:left !important;">編製財務報告類型</th>
<td class="lColor" colspan="3" style="text-align:left !important;">
●合併○個別
</td>
</tr>
</table>

In [4]:
# 第一個table

# 取得資料
data = [th.text.replace('\n', '') for th in table[0].find('tr').find_all('td')]
# 清理資料
out = ["".join(i.split()) for i in data ]
# 處理欄位
head = [th.text.replace('\n', '') for th in table[0].find('tr').find_all('th') if th.text != "本公司"]

In [5]:
# 第二個table

# 取得資料
data2 = [th.text.replace('\n', '') for th in table[1].find_all('td')]
# 清理資料
out2 = ["".join(i.split()) for i in data2 ]
# 處理欄位
head2 = [th.text.replace('\n', '') for th in table[1].find_all('th')  if "本公司" not in th.text]

In [6]:
out2

['', '', '', '●合併○個別']

In [7]:
head2

['月制會計年度（空白表曆年制）', '之前採', '月制會計年度', '編製財務報告類型']

In [8]:
# 合併兩段table資料與欄位
columns = head + head2
rowdata = out + out2

In [9]:
# list在dataframe預設為column資料，所以傳入時，用[]強制為行資料
df = pd.DataFrame([rowdata],columns=columns)
df

,股票代號,產業類別,外國企業註冊地國,公司名稱,總機,地址,董事長,總經理,發言人,發言人職稱,...,投資人關係聯絡電話,投資人關係電子郵件,公司網站內利害關係人專區網址,變更前名稱,變更前簡稱,公司名稱變更核准日期,月制會計年度（空白表曆年制）,之前採,月制會計年度,編製財務報告類型
0,2330,半導體業,－,台灣積體電路製造股份有限公司,03-5636688,新竹科學園區力行六路8號,劉德音,總裁:魏哲家,黃仁昭,副總經理暨財務長,...,03-568-2089,jeff_su@tsmc.com,https://esg.tsmc.com/csr/ch/CSR/stakeholder.html,,,0,,,,●合併○個別


In [10]:
# 存檔
df.to_csv('2330_05.csv',index=False)